<a href="https://colab.research.google.com/github/Crisitunity-Lab/ARDC-Project/blob/main/Prototype_Falcon_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Falcon 7B
This notebook builds on [this](https://github.com/andrewgcodes/FalconStreaming/blob/main/Falcon7B_Instruct_Streaming.ipynb) notebook created by Andrew Gao.


In [2]:
!pip install git+https://www.github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate
!pip install bitsandbytes
!pip install einops

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import torch

  Cloning https://www.github.com/huggingface/transformers to /tmp/pip-req-build-cia0dgbi
  Running command git clone --filter=blob:none --quiet https://www.github.com/huggingface/transformers /tmp/pip-req-build-cia0dgbi
  Resolved https://www.github.com/huggingface/transformers to commit 0afa5071bd84e44301750fdc594e33db102cf374
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-_n_0cxd6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-_n_0cxd6
  Resolved https://github.com/huggingface/accelerate to commit 739b135f8367becb67ffaada12fe76e3aa60fefd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
model_path="tiiuae/falcon-7b-instruct"

config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, load_in_4bit=True, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")

In [ ]:
input_text = "In a table, tell me the sentiment, topic, subtopic, location and the crisis for which this tweet relates: RT @CBCAlerts: Canmore, Alta. declares state of emergency due to flooding  - with some residents being moved to community centre #Alberta"